# Introdução ao xarray

`xarray` é particularmente útil para manipular, analisar e visualizar dados multidimensionais que têm coordenadas rotuladas e metadados. Ele se integra bem com outras bibliotecas do ecossistema científico Python, como `NumPy`, `pandas` e `Matplotlib`.

### Instalação

Para instalar o xarray, você pode usar o pip:

`pip install xarray`

Ou instalar pelo conda:

`conda install conda-forge::xarray`

#### Importando xarray

Para começar a usar o xarray, você precisa importá-lo:

In [ ]:
import xarray as xr
import numpy as np
import scipy

### Estruturas de Dados Básicas

O `xarray` possui duas estruturas de dados principais: `DataArray` e `Dataset`.

### DataArray

Um `DataArray` é um array N-dimensional com coordenadas associadas a cada dimensão.

In [ ]:
# Criando um DataArray simples
data = np.random.rand(4, 3)
data_array = xr.DataArray(data, dims=["x", "y"], coords={"x": [10, 20, 30, 40], "y": ["a", "b", "c"]})
data_array

#### Manipulação de Dados

Seleção e Indexação

Você pode selecionar dados usando métodos como `sel`, `isel` e fatiamento padrão como o `slice`.

In [ ]:
# Selecionando dados com coordenadas
data_array.sel(x=20, y="b")


In [ ]:
# Selecionando dados com índices
data_array.isel(x=1, y=1)


In [ ]:
# Fatiamento
data_array.sel(x=slice(10, 30))


#### Operações Matemáticas

In [ ]:
# Operações element-wise
data_array + 10



In [ ]:
data_array * 2


#### Aplicando funções matemáticas

In [ ]:
np.sin(data_array)

In [ ]:
data_array.mean(dim="x")

### Dataset

Um `Dataset` é um contêiner de múltiplos `DataArrays`, análogos a um DataFrame do pandas.

In [ ]:
# Criando um Dataset
data1 = xr.DataArray(np.random.rand(4, 3), dims=["x", "y"], coords={"x": [10, 20, 30, 40], "y": ["a", "b", "c"]})
data2 = xr.DataArray(np.random.rand(4), dims=["x"], coords={"x": [10, 20, 30, 40]})

dataset = xr.Dataset({"data1": data1, "data2": data2})
dataset


#### Seleção e Indexação em `Dataset`

Você pode selecionar dados de um Dataset de forma semelhante a um `DataArray`.

In [ ]:
# Selecionando um DataArray específico
dataset["data1"]

In [ ]:
# Selecionando dados com coordenadas
dataset.sel(x=20)

In [ ]:
# Selecionando dados com índices
dataset.isel(x=1)

#### Adicionando e Removendo Variáveis em Dataset

In [ ]:
# Adicionando uma variável
dataset["data3"] = ("x", [7, 8, 9, 10])
dataset



In [ ]:
# Removendo uma variável
dataset = dataset.drop_vars("data3")
dataset

### Aplicando Funções a Datasets

In [ ]:
dataset.mean(dim="x")

### Leitura e Escrita de Arquivos

`xarray` suporta leitura e escrita de vários formatos de arquivo, como NetCDF, HDF5, GRIB, entre outros.

In [ ]:
# Salvando um Dataset como um arquivo NetCDF
dataset.to_netcdf("dataset.nc")

In [ ]:
# Carregando um Dataset de um arquivo NetCDF
loaded_dataset = xr.open_dataset("dataset.nc")
print(loaded_dataset)

### Visualização de Dataset

Você pode visualizar variáveis de um `Dataset` de forma semelhante a um `DataArray`.

In [ ]:
# Plotando uma variável de um Dataset
dataset["data1"].plot()


## Gerando um `DataArray` como se fosse uma variavel de temperatura em Kelvin

Primeiro vamos criar um NumPy array com valores próximos a um campo de temperatura em Kelvin

In [ ]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

Uma forma para transformar a variável `data` que criamos com o NumPy array em um `DataArray` é usar o método `DataArray` do Xarray

In [ ]:
temp = xr.DataArray(data)
temp

Observe duas coisas:

 - Como os arrays NumPy não têm nomes de dimensão, nosso novo DataArray assume nomes de dimensão de espaço reservado, neste caso dim_0, dim_1 e dim_2. Em nosso próximo exemplo, demonstramos como adicionar nomes de dimensão mais significativos.

 - Se você estiver visualizando esta página como um Jupyter Notebook, executar o exemplo acima gera uma exibição rica dos dados contidos em nosso DataArray. Esta exibição vem com muitas maneiras de explorar os dados; por exemplo, clicar no símbolo do array expande ou recolhe a exibição de dados.

### Alterando os nomes das dimensões e coordenadas

Uma das principais características do Xarray está no uso de dimensões nomeadas. Para aproveitar ao máximo essa funcionalidade, é importante fornecer nomes mais significativos de dimensão. Isso pode ser feito ao criar um `DataArray`, passando uma lista ordenada de nomes para o método `DataArray` através do argumento de palavra-chave `dims`

In [ ]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

Este `DataArray` já oferece vantagens em comparação com um array do NumPy, pois inclui nomes para cada uma das suas dimensões (ou eixos, na terminologia do NumPy). Uma melhoria adicional é a possibilidade de associar arrays de coordenadas aos dados quando o `DataArray` é criado. No exemplo a seguir, vamos demonstrar como criar arrays do NumPy que representam os valores das coordenadas para cada dimensão do `DataArray` e como vincular esses arrays de coordenadas aos dados dentro do `DataArray`.

Criando as coordenadas de espaço e tempo

Neste exemplo, vamos usar o Pandas para criar o array do tempo. Este array será usado em um exemplo posterior para adicionar ao `DataArray` como uma coordenada nomeada, denominada time.

In [ ]:
import pandas as pd
times = pd.date_range('2024-01-01', periods=5)
times

Criando coordenadas para a latitude e longitude

In [ ]:
lons = np.linspace(-44, -42.5, 4)
lats = np.linspace(-22.5, -23.5, 3)

### Criando o `DataArray` com as informações completas

Neste exemplo, estamos criando um novo `DataArray`. Assim como no exemplo anterior, utilizamos o argumento de palavra-chave `dims` para definir os nomes das dimensões; no entanto, desta vez, também incluímos os arrays de coordenadas por meio do argumento de palavra-chave `coords`:

In [ ]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

Adicionando atributos

In [ ]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

##### Problemática com a preservação dos atributos 

Quando realizamos uma operação matemática em um `DataArray`, os arrays de coordenadas permanecem associados a ele, mas os metadados de atributo são eliminados. Isso ocorre porque esses atributos podem não representar corretamente os metadados após uma operação aritmética qualquer.

No exemplo a seguir, vamos converter os valores do `DataArray` de Kelvin para graus Celsius. Preste atenção aos atributos na visualização avançada do Jupyter abaixo. (Se você não estiver visualizando esta página como um notebook do Jupyter, consulte a documentação do Xarray para aprender como acessar os atributos.)

In [ ]:
temp_in_celsius = temp - 273.15
temp_in_celsius

### `Dataset`

Como comentamos, um `Dataset` é um container de `DataArray`.

No próximo exemplo, vamos criar um `DataArray` correspondente a Pressão Atmosférica de forma bem similar ao que utilizamos anteriormente.

In [ ]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

Antes de criarmos um objeto `Dataset`, é necessário nomear cada um dos objetos `DataArray` que serão incorporados a ele.

Para atribuir nomes aos `DataArrays` que faremos parte do nosso `Dataset`, podemos configurar um dicionário em Python, como ilustrado no próximo exemplo. Em seguida, podemos passar esse dicionário para o método `Dataset` usando o argumento de palavra-chave `data_vars`, o que resulta na criação de um novo `Dataset` que inclui ambos os `DataArrays`.

In [ ]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

### Acessando as variáveis dentro do `Dataset`

Existem algumas formas para acessar a variável dentro de um `Dataset`

In [ ]:
ds.Pressure

Ou...

In [ ]:
ds['Pressure']

### Seccionando um `Dataset`

Novamente, existem diversas formas de seccionar um parte de um `Dataframe`. 

O primeiro exemplo é uma forma similar a um NumPy array para janeiro de 2024

In [ ]:
indexed_selection = temp[1, :, :]  
indexed_selection

Outra forma é utilizando o comando `.sel()` para selecionar o mesmo período

In [ ]:
named_selection = temp.sel(time='2024-01-02')
named_selection

Esse método gera o mesmo resultado que a seleção por índice, mas oferece algumas vantagens:
 - Não precisamos ter conhecimento sobre como o array foi criado ou armazenado.
 - Nosso código não depende do número de dimensões envolvidas.
 - O código é muito mais claro.

## Conclusão

Este tutorial básico deve fornecer uma introdução suficiente para começar a usar o xarray para manipulação e análise de dados multidimensionais. Para mais informações e funcionalidades avançadas, recomendo consultar a [documentação oficial do xarray](https://docs.xarray.dev/en/stable/).